### Basic training and testing

In [1]:
using MLJ
using DataFrames

task = load_boston()
X, y = task()

# optional:
X = DataFrame(X) # or any other tabular format supported by Table.jl 

train, test = partition(eachindex(y), 0.7); # 70:30 split

┌ Info: Recompiling stale cache file /Users/anthony/.julia/compiled/v1.0/MLJ/rAU56.ji for MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1190


A *model* is a container for hyperparameters:

In [2]:
knn_model=KNNRegressor(K=10)

# KNNRegressor{Float64} @ 5…48: 
target_type             =>   Float64
K                       =>   10
metric                  =>   euclidean (generic function with 1 method)
kernel                  =>   reciprocal (generic function with 1 method)



Wrapping the model in data creates a *machine* which will store training outcomes (called *fit-results*):

In [3]:
knn = machine(knn_model, X, y)

# Machine{KNNRegressor{Float64}} @ 1…16: 
model                   =>   KNNRegressor{Float64} @ 5…48
fitresult               =>   (undefined)
cache                   =>   (undefined)
args                    =>   (omitted Tuple{DataFrame,Array{Float64,1}} of length 2)
report                  =>   (undefined)
rows                    =>   (undefined)



Training on the training rows and evaluating on the test rows:

In [4]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(y[test], yhat)

┌ Info: Training Machine{KNNRegressor{Float64}} @ 1…16.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93


8.090639098853249

Or, in one line:

In [5]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7))

8.090639098853249

Changing a hyperparameter and re-evaluating:

In [6]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))

8.41003854724935

### Systematic tuning as a model wrapper

A simple example of a composite model is a homogeneous ensemble. Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [7]:
ensemble_model = EnsembleModel(atom=knn_model, n=20) 

# DeterministicEnsembleModel @ 1…29: 
atom                    =>   KNNRegressor{Float64} @ 5…48
weights                 =>   0-element Array{Float64,1}
bagging_fraction        =>   0.8
rng_seed                =>   0
n                       =>   20
parallel                =>   true



Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of our models is a field of the other, we have nested hyperparameters:

In [8]:
params(ensemble_model) # a named tuple (nested)

(atom = (target_type = Float64, K = 20, metric = MLJ.KNN.euclidean, kernel = MLJ.KNN.reciprocal), weights = Float64[], bagging_fraction = 0.8, rng_seed = 0, n = 20, parallel = true)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [9]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = (atom = (K = K_range,), bagging_fraction = B_range)

(atom = (K = NumericRange @ 1…30,), bagging_fraction = NumericRange @ 1…09)

Now we choose a tuning strategy, and a resampling strategy (for estimating performance), and wrap these strategies around our ensemble model to obtain a new model:

In [10]:
tuning = Grid(resolution=8)
resampling = CV(nfolds=6)

tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

# DeterministicTunedModel @ 3…75: 
model                   =>   DeterministicEnsembleModel @ 1…29
tuning                  =>   Grid @ 9…00
resampling              =>   CV @ 2…48
measure                 =>   nothing
operation               =>   predict (generic function with 19 methods)
nested_ranges           =>   (omitted NamedTuple{(:atom, :bagging_fraction),Tuple{NamedTuple{(:K,),Tuple{MLJ.NumericRange{Int64,Symbol}}},MLJ.NumericRange{Float64,Symbol}}})
full_report             =>   true



Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [11]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Warning: No measure specified. Using measure=MLJ.rms. 
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:82
┌ Info: Training Machine{MLJ.DeterministicTunedMo…} @ 1…98.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/machines.jl:93
Searching a 64-point grid for best model: 100%[=========================] Time: 0:00:46
┌ Info: Training best model on all supplied data.
└ @ MLJ /Users/anthony/Dropbox/Julia7/MLJ/src/tuning.jl:142


In [12]:
fp = fitted_params(tuned_ensemble)

(best_model = DeterministicEnsembleModel @ 4…59,)

In [13]:
@show fp.best_model.bagging_fraction
@show fp.best_model.atom.K;

(fp.best_model).bagging_fraction = 0.5
((fp.best_model).atom).K = 52
